In [1]:
using Distributed
addprocs(1)

1-element Array{Int64,1}:
 2

In [2]:
Distributed.myid()

1

In [3]:
workers()

1-element Array{Int64,1}:
 2

In [4]:
res = @spawnat 2 myid()

Future(2, 1, 3, nothing)

In [5]:
fetch(res)

2

In [6]:
remote_f= function(s::Int=3)
    println("Worker $(myid()) will sleep for $s seconds")
    sleep(s)
    val=rand(1:1000)
    println("Completed worker $(myid()) - return $val")
    return val
end

#5 (generic function with 2 methods)

In [7]:
@fetchfrom 2 remote_f(4)

      From worker 2:	Worker 2 will sleep for 4 seconds
      From worker 2:	Completed worker 2 - return 123


123

In [8]:
function run_timeout(timeout::Int, f::Function, params...)    
    wid = addprocs(1)[1]
    result = RemoteChannel(()->Channel{Tuple}(1));
    @spawnat wid put!(result, (f(params...), myid()))
    res = nothing
    time_elapsed = 0.0
    while time_elapsed < timeout && !isready(result)
        sleep(0.25)
        time_elapsed += 0.25
    end
    if !isready(result)
        println("Not completed! Computation at $wid will be terminated!")  
    else
        res = take!(result)
    end
    rmprocs(wid);
    return res
end

run_timeout (generic function with 1 method)

In [9]:
run_timeout(3, remote_f, 2)

      From worker 3:	Worker 3 will sleep for 2 seconds
      From worker 3:	Completed worker 3 - return 94


(94, 3)

In [10]:
run_timeout(3, remote_f, 10)

      From worker 4:	Worker 4 will sleep for 10 seconds
Not completed! Computation at 4 will be terminated!


In [11]:
workers()

1-element Array{Int64,1}:
 2

In [12]:
using Distributed
@everywhere function myF2(); println("myF2 ", myid()); end;
@spawnat workers()[end] myF2();

      From worker 2:	myF2 2


In [13]:
hello() = println("hello");

In [14]:
@fetchfrom 2 hello()

RemoteException: On worker 2:
UndefVarError: #hello not defined
deserialize_datatype at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:1186
handle_deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:775
deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:722
handle_deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:782
deserialize_global_from_main at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:722
#5 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/clusterserialize.jl:72 [inlined]
foreach at ./abstractarray.jl:1920
deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/clusterseriali

In [15]:
f_lambda = () -> hello();

In [16]:
f_lambda()

hello


In [17]:
@fetchfrom 2 f_lambda()

RemoteException: On worker 2:
UndefVarError: #hello not defined
deserialize_datatype at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:1186
handle_deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:775
deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:722
handle_deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:782
deserialize_global_from_main at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Serialization/src/Serialization.jl:722
#5 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/clusterserialize.jl:72 [inlined]
foreach at ./abstractarray.jl:1920
deserialize at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Distributed/src/clusterseriali

In [18]:
@everywhere hello() = println("hello")

In [19]:
@fetchfrom 2 f_lambda()

      From worker 2:	hello
